In [0]:
# ms-python.python added
import os
try:
	os.chdir("/home/daniel/CG_results_20191004/")
	print(os.getcwd())
except:
	pass


In [0]:
import numpy as np
import pandas as pd
import re



In [0]:
file_name ="CG_overall_20191004.csv" 
data = pd.read_csv(file_name)
match = re.search(r'.*\_(\d{4})(\d{2})(\d{2})\.csv',file_name)
year = match.group(1)
month = match.group(2)
day = match.group(3)
print(type(day))



In [0]:
data['gap'] = (data['global_upper_bound'] - data['global_lower_bound']
               )/(data['global_lower_bound'] + 0.00001)
data['opt'] = data['global_lower_bound'] == data['global_upper_bound']
data['reduction'] = (data['first_size_graph'] - data['size_after_reduced_cost'])/(data['first_size_graph'] + 0.000001)
lst = []
import re
for l in data['NameInstance']:
    test = re.search(r'.*\_(\d+)', l).group(1)
    lst.append(int(test))

se = pd.Series(lst)
data['Id'] = se.values



In [0]:
summary_grouped = data.groupby(['pricing_solver','n','m'])



In [0]:
aggregation = {"tot_lb": {np.max, np.mean},
               "gap": {np.max, np.mean},
               "first_size_graph": {np.max, np.mean},
               "opt": np.sum,
               "reduction": {np.max,np.mean},
              "tot_cputime" : {np.max, np.mean}}


In [0]:
summary_write = summary_grouped.agg(aggregation).pivot_table(index=['n', 'm'], values=[
                              'tot_lb', 'gap', 'first_size_graph','reduction','opt'], columns=['pricing_solver'])
summary_write.columns.set_levels(['AFFS','AFFC','AFBS','AFBC','AFZFS','AFZFC','AFZBS','AFZBC','TI','ATI'],level=2,inplace=True)
summary_write.columns = ["_".join(x) for x in summary_write.columns.ravel()]
summary_write.to_csv("CG_summary_"+year+month+day+".csv")
summary_write


In [0]:
all_instances = data.pivot_table(values=['tot_lb', 'gap', 'first_size_graph','reduction','opt','rel_error','nb_generated_col','global_lower_bound','global_upper_bound','tot_cputime'], index=['n','m','Id'], columns=['pricing_solver'])
all_instances.columns.set_levels(['AFFS','AFFC','AFBS','AFBC','AFZFS','AFZFC','AFZBS','AFZBC','TI','ATI'],level=1,inplace=True)
all_instances.columns = ["_".join(x) for x in all_instances.columns.ravel()]
all_instances.to_csv("CG_allinstances_"+year+month+day+".csv")

